## Load MCP config

In [ ]:
import json
import os

def load_mcp_servers(config_path):
    """
    Load MCP server definitions from a JSON config file.
    Expects a top-level 'mcpServers' dict in the config.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        config = json.load(f)
    servers = config.get("mcpServers", {})
    # Optionally add default transports if missing
    for name, server in servers.items():
        if "command" in server and "transport" not in server:
            server["transport"] = "stdio"
        if "url" in server and "transport" not in server:
            server["transport"] = "streamable_http"
    return servers

mcp_servers = load_mcp_servers("../config/mcp_servers.json")
mcp_servers

## Load tools

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(mcp_servers)
mcp_tools = await client.get_tools()

len(mcp_tools)

## Agent wiht MCP tools

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from tools import draw_mermaid_png


agent = create_react_agent(
    model="openai:gpt-4o-mini",
    prompt="You are a GitHub Assistant that helps users manage their GitHub repositories and workflows.",
    tools=mcp_tools,
    checkpointer=InMemorySaver(),
)

draw_mermaid_png(agent)

## Adding HITL to each tool

### Create Wrapper

In [ ]:
from typing import Callable
from langchain_core.tools import BaseTool, tool
from langchain_core.runnables import RunnableConfig
from langgraph.types import interrupt

def add_approval(main_tool: Callable | BaseTool) -> BaseTool:
    """Wrap a tool to support human-in-the-loop review."""
    if not isinstance(main_tool, BaseTool):
        main_tool = tool(main_tool)

    @tool(  
        main_tool.name,
        description=main_tool.description,
        args_schema=main_tool.args_schema
    )
    async def call_main_tool_with_hitl(config: RunnableConfig, **tool_input):
        decision = interrupt({
            "awaiting": main_tool.name,
            "args": tool_input
        })

        # tool approved
        if isinstance(decision, dict) and decision.get("approved"):
            result = await main_tool.ainvoke(tool_input, config)
            return result

        # tool rejected
        return "Cancelled by human. Continue without executing that tool and provide next steps."
        

    return call_main_tool_with_hitl

### Wrapping all tools with HITL (could be a spec algo that wrap only risky tools)

In [ ]:
tools_with_hitl = [add_approval(tool) for tool in mcp_tools]
len(tools_with_hitl)

### New Agent with all the tools wrapped into HITL

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from tools import draw_mermaid_png


agent = create_react_agent(
    model="openai:gpt-4o-mini",
    prompt="You are a GitHub Assistant that helps users manage their GitHub repositories and workflows.",
    tools=tools_with_hitl,
    checkpointer=InMemorySaver(),
)

draw_mermaid_png(agent)

### some basic stuff

In [ ]:
def print_tool_approval(payload):
    tool = payload.get("awaiting", "unknown_tool")
    args = payload.get("args", {})

    print("—-- Approval needed —--")
    print(f"Tool: {tool}")

    if isinstance(args, dict) and args:
        print("Parameters:")
        for k, v in args.items():
            print(f"  - {k}: {v}")
    elif args:
        print(f"Parameters: {args}")
    else:
        print("No parameters.")

## Small testing

In [ ]:
import uuid
from textwrap import dedent
from langchain_core.messages import HumanMessage

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())
    }
}

prompt = dedent("""
Can you check my account on GitHub and look at my recent work on the langgraph-hitl-fastapi-demo repo? 
I want to understand what I've been working on lately.
So collect information about all my recent activity and provide a brief overview in natural human language 
about my recent work.
""")

response = await agent.ainvoke({"messages": [HumanMessage(content=prompt)]}, config)
for message in response['messages']:
    message.pretty_print()

In [ ]:
"__interrupt__" in response

In [ ]:
interrupts = response["__interrupt__"]
print_tool_approval(interrupts[0].value)

In [ ]:
from langgraph.types import Command

response = await agent.ainvoke(Command(resume={"approved": True}), config=config)
for message in response['messages']:
    message.pretty_print()